<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [4]:
%%sql
SELECT title
FROM movies

 * sqlite:///movies.sqlite
Done.


title
Pulp Fiction
The Dark Knight
The Godfather
The Lord of the Rings: The Fellowship of the Ring
The Shawshank Redemption


+ Show the performance dates for one of the movies.

In [5]:
%%sql
SELECT screen_date
FROM tickets

 * sqlite:///movies.sqlite
Done.


screen_date
2021-02-11
2021-02-11
2021-02-11
2021-02-12


+ Show all data concerning performances at a given theatere
  on a given date.

In [6]:
%%sql
SELECT *
FROM performances
WHERE screen_date = '2021-02-11' and th_name = 'Lund'


 * sqlite:///movies.sqlite
Done.


screen_date,start_time,th_name,title,year
2021-02-11,21:00,Lund,The Godfather,1972
2021-02-11,19:30,Lund,The Shawshank Redemption,1994


+ List all customers

In [7]:
%%sql
SELECT usr_name, full_name
FROM customers


 * sqlite:///movies.sqlite
Done.


usr_name,full_name
lemur,Anna Ek
wildcat,Jonny Balle
bison,Kim Jong Un
greyhound,Angela merkel


+ List all tickets

In [8]:
%%sql
SELECT t_id, title, usr_name
FROM tickets


 * sqlite:///movies.sqlite
Done.


t_id,title,usr_name
72e9fb436964b5878c1ad59edeec91d7,The Shawshank Redemption,lemur
bece9c9e2982f3fb5f521a3da9edc5f7,The Shawshank Redemption,wildcat
a697d8413c69681dcf526842547a8297,The Shawshank Redemption,bison
b411df3957d6c349486da3dcfac354cb,The Shawshank Redemption,greyhound


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [9]:
%%sql
INSERT 
INTO tickets(t_id, th_name, title, year, screen_date, start_time, usr_name)
VALUES (lower(hex(randomblob(16))), 'Lund', 'The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'wildcat')

 * sqlite:///movies.sqlite
1 rows affected.


[]

In [14]:
%%sql
INSERT
INTO tickets(t_id, th_name, title, year, screen_date, start_time, usr_name)
VALUES(lower(hex(randomblob(16))), 'Lund','The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'lemur')

 * sqlite:///movies.sqlite
(sqlite3.OperationalError) foreign key mismatch - "tickets" referencing "performances"
[SQL: INSERT INTO tickets(t_id, th_name, title, year, screen_date, start_time, usr_name)
VALUES(lower(hex(randomblob(16))), 'Lund','The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'lemur')]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2021), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [14]:
%%sql
SELECT t_id
FROM tickets
WHERE  rowid = last_insert_rowid();


 * sqlite:///movies.sqlite
Done.


t_id
23403f8db81f2c9b4377d6ce2b1f4fa8


+ Try to insert two movie theaters with the same name (this
  should fail).

In [15]:
%%sql
INSERT
INTO theatres(th_name, capacity)
VALUES('Lund', 150)

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theatres.th_name
[SQL: INSERT INTO theatres(th_name, capacity)
VALUES('Lund', 150)]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [16]:
%%sql
INSERT
INTO performances(title, year, screen_date, start_time, th_name)
VALUES('The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'empty')


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO performances(title, year, screen_date, start_time, th_name)
VALUES('The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'empty')]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [17]:
%%sql
INSERT
INTO tickets(t_id, th_name, title, year, screen_date, start_time, usr_name)
VALUES(lower(hex(randomblob(16))), 'Lund','The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'None')


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets(t_id, th_name, title, year, screen_date, start_time, usr_name)
VALUES(lower(hex(randomblob(16))), 'Lund','The Shawshank Redemption', 1994, '2021-02-11', '19:30', 'None')]
(Background on this error at: http://sqlalche.me/e/13/gkpj)